In [1]:
using Plots,LinearAlgebra

# Proyecto Final-Trompo que gira sobre su eje

#### Física Computacional. Grupo 8266.

#### Facultad de Ciencias. Universidad Nacional Autónoma de México

 En este codigo desarrollamos una simulación de un trompo usando las técnicas vistas en clase y un poco más